### Load dependencies

In [4]:
import json, os, uuid
from openai import AzureOpenAI
from azure.cosmos import CosmosClient
from dotenv import load_dotenv

load_dotenv()


True

### Function to create embeddings

In [5]:
from tenacity import retry, wait_random_exponential, stop_after_attempt  

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(10))
def generate_embeddings(openai_client, text):
    """
    Generates embeddings for a given text using the OpenAI API v1.x
    """
    
    return openai_client.embeddings.create(
        input = text,
        model= os.getenv("AZURE_OPENAI_EMBEDDING_MODEL")
    ).data[0].embedding

### Step 1: Init Cosmos DB

In [6]:
COSMOS_DB_ENDPOINT = os.getenv('AZURE_COSMOSDB_NOSQL_ENDPOINT')
COSMOS_DB_KEY = os.getenv('AZURE_COSMOSDB_NOSQL_KEY')
DATABASE_NAME = os.getenv('AZURE_COSMOSDB_NOSQL_DATABASE_NAME')
PRODUCTS_CONTAINER_NAME = os.getenv('AZURE_COSMOSDB_NOSQL_PRODUCTS_CONTAINER_NAME')
VECTORS_CONTAINER_NAME = os.getenv('AZURE_COSMOSDB_NOSQL_VECTORS_CONTAINER_NAME')

client = CosmosClient(COSMOS_DB_ENDPOINT, COSMOS_DB_KEY)
database = client.get_database_client(DATABASE_NAME)
products_container = database.get_container_client(PRODUCTS_CONTAINER_NAME)
vectors_container = database.get_container_client(VECTORS_CONTAINER_NAME)

### Step 2: Initialize Azure OpenAI client

In [7]:
openai_client = AzureOpenAI(
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version = os.getenv("AZURE_OPENAI_API_VERSION"),  
    azure_endpoint =os.getenv("AZURE_OPENAI_ENDPOINT") 
)

### Step 3: Load products from json file

In [8]:
with open('product.json') as file:
  products = json.load(file)

### Step 4: Write to 2 Cosmos DB containers

In [9]:
# Write product and vector in separate containers (to make it easier for change feed modifications)

print("Writing content to Cosmos DB..")
for p in products:
    productKey = str(uuid.uuid4())
    productJson = json.dumps(p)
    product = {
        "id": productKey,
        "categoryId": p["categoryId"],
        "categoryName": p["categoryName"],
        "sku": p["sku"],
        "name": p["name"],
        "description": p["description"],
        "price": p["price"],
        "tags": p["tags"],
    }
    vector = {
        "id": productKey,
        "embedding": generate_embeddings(openai_client, str(productJson))
    }
    products_container.create_item(product)
    vectors_container.create_item(vector)        
    print(f"Product {p['name']} inserted successfully.")

Writing content to Cosmos DB..
Product LL Road Seat/Saddle inserted successfully.
Product Touring-1000 Blue, 50 inserted successfully.
Product ML Road Pedal inserted successfully.
Product Mountain Bottle Cage inserted successfully.
Product HL Road Tire inserted successfully.
Product ML Mountain Seat/Saddle inserted successfully.
Product Road Bottle Cage inserted successfully.
Product ML Mountain Tire inserted successfully.
Product Touring Tire Tube inserted successfully.
Product Touring-1000 Blue, 46 inserted successfully.
Product Touring-1000 Yellow, 46 inserted successfully.
Product HL Touring Seat/Saddle inserted successfully.
Product Touring-1000 Blue, 60 inserted successfully.
Product Women's Tights, S inserted successfully.
Product LL Bottom Bracket inserted successfully.
Product Touring-2000 Blue, 60 inserted successfully.
Product Road Tire Tube inserted successfully.
Product ML Mountain Pedal inserted successfully.
Product Touring-3000 Blue, 44 inserted successfully.
Product LL

In [10]:
result = products_container.query_items(query = 'SELECT VALUE COUNT(1) FROM c', enable_cross_partition_query=True)
total_count = result.next()
print(f"There are {total_count} products in the container")

There are 295 products in the container
